In [1]:
from pyspark.sql import functions as F

import pandas as pd
import numpy as np

In [2]:
sqlContext.sql("set spark.sql.shuffle.partitions = 300")

DataFrame[key: string, value: string]

In [3]:
sqlContext.sql("show tables in ignite").collect()

[Row(tableName=u'all_hashed', isTemporary=False),
 Row(tableName=u'contest_cr_compressed', isTemporary=False),
 Row(tableName=u'contest_data_sample', isTemporary=False),
 Row(tableName=u'contest_data_stamped', isTemporary=False),
 Row(tableName=u'contest_savm_compressed', isTemporary=False),
 Row(tableName=u'contest_sol_end_customer_compressed', isTemporary=False),
 Row(tableName=u'cr_hashed', isTemporary=False),
 Row(tableName=u'lsh_pairs', isTemporary=False),
 Row(tableName=u'savm_candidate_gen', isTemporary=False),
 Row(tableName=u'savm_hashed', isTemporary=False),
 Row(tableName=u'string_to_string', isTemporary=False),
 Row(tableName=u'test', isTemporary=False)]

In [ ]:
contest_data = sqlContext.sql("select * from ignite.contest_data_stamped").cache()
so_end_customer = sqlContext.sql('select * from edso_ignite.contest_so_end_customer').repartition(200).cache()
sol_end_customer = sqlContext.sql('select * from edso_ignite.contest_sol_end_customer').repartition(200).cache()
party_info = sqlContext.sql("select * from edso_ignite.contest_party_info").repartition(200).cache()
end_customer = sqlContext.sql('select * from edso_ignite.contest_end_customer').repartition(200).cache()
cr = sqlContext.sql('select * from edso_ignite.contest_cr').repartition(200).cache()
savm = sqlContext.sql("select * from edso_ignite.contest_savm").repartition(200, 'party_id').cache()

In [ ]:
def drop_columns(df, columns):
    return df.select([c for c in df.columns if c not in columns])

In [ ]:
party_values = [
    'end_customer_party_ssot_party_id_int_sav_party_id',
    'prior_party_ssot_party_id_int_sav_party_id',
    'sol_branch_party',
    'sol_gu_party',
    'sol_hq_party',
    'order_level_branch_party',
    'order_level_gu_party',
    'order_level_hq_party',
    'line_level_branch_party',
    'line_level_gu_party',
    'line_level_hq_party',
    'ship_to_branch_party',
    'ship_to_gu_party',
    'ship_to_hq_party',
    'bill_to_branch_party',
    'bill_to_gu_party',
    'bill_to_hq_party',
    'sold_to_branch_party',
    'sold_to_gu_party',
    'sold_to_hq_party'
]

In [ ]:
party_joined = sqlContext.sql('select * from ignite.party_expansion')

In [ ]:
all_parties = party_joined.map(lambda x : (x.id, list(set([
            x.end_customer_party_ssot_party_id_int_sav_party_id,
            x.prior_party_ssot_party_id_int_sav_party_id,
            x.pppk_party,
            x.sol_branch_party,
            x.sol_gu_party,
            x.sol_hq_party,
            x.order_level_branch_party,
            x.order_level_gu_party,
            x.order_level_hq_party,
            x.line_level_branch_party,
            x.line_level_gu_party,
            x.line_level_hq_party,
            x.ship_to_branch_party,
            x.ship_to_gu_party,
            x.ship_to_hq_party,
            x.bill_to_branch_party,
            x.bill_to_gu_party,
            x.bill_to_hq_party,
            x.sold_to_branch_party,
            x.sold_to_gu_party,
            x.sold_to_hq_party
        ])))).flatMapValues(lambda x : x).toDF(['id', 'party_id']).repartition(1000, 'id').cache()

In [26]:
# really should make this only 1 -> 2 not have 2 -> 1 as well....
lsh = sqlContext.sql("select * from ignite.lsh_pairs_2")

In [6]:
lsh.count()

37959240

In [5]:
lsh.select(['party_id_1']).distinct().count()

21611027

In [9]:
lsh.where(F.col('party_id_1') == 37578708).collect()

[Row(party_id_1=37578708.0, party_id_2=44150932.0)]

In [13]:
all_parties.count()

5110563

In [27]:
lsh_1 = lsh.select([
        F.col('party_id_1').alias('party_id'),
        F.col('party_id_2').alias('party_id_candidate')
    ])

lsh_2 = lsh.select([
        F.col('party_id_2').alias('party_id'),
        F.col('party_id_1').alias('party_id_candidate')
    ])

builder1 = all_parties.join(lsh_1, 'party_id').drop_duplicates().repartition(1000, 'party_id')
builder2 = all_parties.join(lsh_2, 'party_id').drop_duplicates().repartition(1000, 'party_id')
expanded = builder1.unionAll(builder2).cache()

In [21]:
expanded.count()

824932724

In [9]:
expanded.take(50)

[Row(party_id=8500631, id=8589946146, party_id_candidate=222780809.0),
 Row(party_id=273717631, id=266287980750, party_id_candidate=225000924.0),
 Row(party_id=273717631, id=266287980730, party_id_candidate=194758032.0),
 Row(party_id=38631, id=377957131730, party_id_candidate=89023837.0),
 Row(party_id=253556631, id=377957130401, party_id_candidate=257772343.0),
 Row(party_id=3946631, id=180388628685, party_id_candidate=162191.0),
 Row(party_id=3946631, id=137438962083, party_id_candidate=162191.0),
 Row(party_id=34631, id=223338313338, party_id_candidate=5935717.0),
 Row(party_id=114777631, id=214748371657, party_id_candidate=277549274.0),
 Row(party_id=133631, id=137438959311, party_id_candidate=206279.0),
 Row(party_id=133631, id=137438963575, party_id_candidate=206279.0),
 Row(party_id=133631, id=137438971911, party_id_candidate=206279.0),
 Row(party_id=133631, id=137438972275, party_id_candidate=206279.0),
 Row(party_id=133631, id=137438964775, party_id_candidate=206279.0),
 Row(

In [28]:
to_merge_savm = savm.select([
        F.col('party_id').alias('party_id_candidate'),
        F.col('sales_acct_id').alias('candidate_sales_acct_id')
    ])

lsh_savm_candidates = expanded.join(to_merge_savm, 'party_id_candidate').select(['id', 'party_id_candidate', 'candidate_sales_acct_id']).drop_duplicates()

In [29]:
lsh_savm_candidates.write.saveAsTable('ignite.lsh_savm_candidates_2', mode = 'overwrite')

In [ ]:
lsh_savm_candidates = sqlContext.sql("select * from ignite.lsh_savm_candidates_")

In [25]:
lsh_savm_candidates.count()

577738815

In [16]:
lsh_savm_candidates.take(1)

[Row(id=111669160840, party_id_candidate=659341.0, candidate_sales_acct_id=203809296.0)]

In [17]:
all_parties_truth = all_parties.join(contest_data.select(['id', F.col('sales_acct_id').alias('truth_sales_acct_id')]), on = 'id')
lsh_matching = all_parties_truth.join(lsh_savm_candidates, on = 'id').where(F.col('truth_sales_acct_id') == F.col('candidate_sales_acct_id')).drop_duplicates().cache()

In [11]:
lsh_matching.count()

203204385

In [16]:
builder = lsh_savm_candidates.join(contest_data.select(['id', F.col('sales_acct_id').alias('truth_sales_acct_id')]), on = 'id')
name_candidates = drop_columns(builder.where(F.col('candidate_sales_acct_id') == F.col('truth_sales_acct_id')), ['party_id']).cache()

In [69]:
name_candidates.select(['id']).distinct().count()

479164

In [18]:
lsh_matching.select(['id']).distinct().count()

588537

In [17]:
id_candidate_gen = sqlContext.sql("select * from ignite.savm_candidate_gen")

In [18]:
builder = id_candidate_gen.join(contest_data.select(['id', F.col('sales_acct_id').alias('truth_sales_acct_id')]), on = 'id')
id_candidates = builder.where(F.col('candidate_sales_acct_id') == F.col('truth_sales_acct_id')).cache()

In [19]:
contest_data.take(1)

[Row(endcustomerlinefixed=u'CALPINE', pppk=113405770, fppk=677259, decision_date_time=u'2015-07-03 05:48:05.0', days_from_create_to_final_decision=20.208333, end_customer_party_ssot_party_id_int_sav_party_id=184948898, prior_party_ssot_party_id_int_sav_party_id=184948898, final_party_ssot_party_id_int_sav_party_id=2649467, type=u'Reviewed_and_modified', priorpartyname=u'CALPINE', finalpartyname=u'CALPINE CORPORATION', sales_order_key=23867682, sales_order_line_key=235827955, so_number_int=100729935, end_customer_key_line_level=52302581, end_customer_key_order_level=14654135, dd_end_customer_type_order_level=u'ACCOUNT_LOCATION                                  ', purchase_order_type_code=u'Resale', purchase_order_number=u'38504904-01', order_datetime=u'2015-06-13 00:05:58.0', oracle_book_datetime=u'2015-06-12 17:38:00.0', transactional_currency_code=u'USD            ', customer_service_rep_name=u'null', cust_svc_rep_team_name=u'null', conversion_type_code=u'Corporate', original_system_re

In [20]:
exact_match = contest_data.select(['id', 'priorpartyname', F.col('sales_acct_id').alias('truth_sales_acct_id')]).join(savm.select(['party_name', F.col('sales_acct_id').alias('candidate_sales_acct_id')]), F.col('priorpartyname') == F.col('party_name'))
exact_match_candidates = exact_match.where(F.col('candidate_sales_acct_id') == F.col('truth_sales_acct_id')).cache()

In [21]:
exact_match_candidates.select(['id']).distinct().count()

584740

In [136]:
exact_match_candidates.count()

0

In [22]:
all_candidates = id_candidates.select(['id']).unionAll(lsh_matching.select(['id'])).unionAll(exact_match_candidates.select(['id'])) \
                    .distinct().withColumn("matching", F.lit(1))

In [28]:
id_candidates

DataFrame[id: bigint, candidate_sales_acct_id: double, truth_sales_acct_id: double]

In [23]:
all_candidates.count()

783466

In [147]:
all_candidates.count()

783466

In [24]:
joined_status = all_parties.join(all_candidates, on = 'id', how = 'left').write.saveAsTable("ignite.candidate_match_status")

In [121]:
joined_status.take(100)

[Row(id=34359743991, party_id=156785794, matching=None),
 Row(id=34359743991, party_id=258022883, matching=None),
 Row(id=34359743991, party_id=12228, matching=None),
 Row(id=34359743991, party_id=173869477, matching=None),
 Row(id=34359743991, party_id=6145705, matching=None),
 Row(id=34359743991, party_id=147183, matching=None),
 Row(id=34359743991, party_id=6320533, matching=None),
 Row(id=34359743991, party_id=29115, matching=None),
 Row(id=34359749991, party_id=211370296, matching=None),
 Row(id=34359749991, party_id=137264, matching=None),
 Row(id=34359749991, party_id=180734435, matching=None),
 Row(id=34359749991, party_id=9219126, matching=None),
 Row(id=34359749991, party_id=145942526, matching=None),
 Row(id=51539609171, party_id=184813954, matching=None),
 Row(id=51539609171, party_id=221424901, matching=None),
 Row(id=51539609171, party_id=148557874, matching=None),
 Row(id=51539609171, party_id=38265556, matching=None),
 Row(id=51539609171, party_id=157246237, matching=No

In [ ]:
savm.where(F.col('party_id') == 37578708).take(1)

In [130]:
contest_data.where(F.col('id') == 154618837251).take(1)

[Row(endcustomerlinefixed=u'KLOCKNER PENTAPLAST GMBH', pppk=76882999, fppk=3640090, decision_date_time=u'2016-03-18 10:09:53.0', days_from_create_to_final_decision=1.208333, end_customer_party_ssot_party_id_int_sav_party_id=139723356, prior_party_ssot_party_id_int_sav_party_id=139723356, final_party_ssot_party_id_int_sav_party_id=2918281, type=u'Reviewed_and_modified', priorpartyname=u'KLOCKNER PENTAPLAST GMBH', finalpartyname=u'KIRKBI AS', sales_order_key=25434223, sales_order_line_key=256363324, so_number_int=102111415, end_customer_key_line_level=71002271, end_customer_key_order_level=71002271, dd_end_customer_type_order_level=u'ACCOUNT_LOCATION                                  ', purchase_order_type_code=u'Managed Service', purchase_order_number=u'8110259096', order_datetime=u'2016-03-17 05:24:46.0', oracle_book_datetime=u'2016-03-17 05:25:28.0', transactional_currency_code=u'USD            ', customer_service_rep_name=u'null', cust_svc_rep_team_name=u'null', conversion_type_code=u